# 木棒台車遊戲測試(CartPole testing)

In [1]:
# 匯入Gymnasium套件，用來建立與操作強化學習環境
import gymnasium as gym

# 載入Gymnasium套件中的木棒台車(CartPole)環境
env = gym.make("CartPole-v1")

# 參數初始化
no = 50            # 比賽回合數(總共模擬50次遊戲)
all_rewards = []   # 每回合的累積報酬會儲存在這個list中
all_steps = []     # 每回合的步數也會儲存在這個list中
total_rewards = 0  # 單一回合累積報酬
total_steps = 0    # 單一回合累積步數

# 環境初始化
observation, info = env.reset()  # 重置環境並取得初始觀測值

# 主迴圈：進行50回合模擬
while no > 0:
    # 隨機選擇一個動作(在CartPole遊戲中是：向左或向右推)
    action = env.action_space.sample()
    total_steps += 1

    # 將動作送進環境，進行下一步模擬
    observation, reward, terminated, truncated, info = env.step(action)

    # done表示這一回合是否結束(超時或失敗)
    done = terminated or truncated

    # 累計該回合的總報酬
    total_rewards += reward

    # 若該回合已結束，重置環境，並記錄該回合的報酬與步數
    if done:
        observation, info = env.reset()    # 重置環境到初始狀態，並傳回額外的資訊
        all_rewards.append(total_rewards)  # 儲存該回合的總報酬
        all_steps.append(total_steps)      # 儲存該回合的總步數
        total_rewards = 0                  # 清空本回合報酬
        total_steps = 0                    # 清空本回合步數
        no -= 1                            # 剩餘回合數減一

# 結束模擬，關閉環境
env.close()

In [2]:
# 顯示每回合的執行結果，包括回合數、累積報酬和結果判定
print('回合\t報酬\t結果')

# 判斷結果：如果步數達到或超過200，視為勝利，否則為失敗
for i, (rewards, steps) in enumerate(zip(all_rewards, all_steps)):
    result = 'Win' if steps >= 200 else 'Loss'
    
    # 輸出回合編號、累積報酬和結果
    print(f'{i}\t{rewards}\t{result}')

回合	報酬	結果
0	18.0	Loss
1	11.0	Loss
2	32.0	Loss
3	38.0	Loss
4	17.0	Loss
5	12.0	Loss
6	16.0	Loss
7	10.0	Loss
8	17.0	Loss
9	16.0	Loss
10	11.0	Loss
11	14.0	Loss
12	13.0	Loss
13	12.0	Loss
14	16.0	Loss
15	26.0	Loss
16	22.0	Loss
17	13.0	Loss
18	27.0	Loss
19	21.0	Loss
20	13.0	Loss
21	17.0	Loss
22	25.0	Loss
23	30.0	Loss
24	39.0	Loss
25	10.0	Loss
26	21.0	Loss
27	13.0	Loss
28	52.0	Loss
29	24.0	Loss
30	30.0	Loss
31	18.0	Loss
32	31.0	Loss
33	13.0	Loss
34	13.0	Loss
35	30.0	Loss
36	16.0	Loss
37	13.0	Loss
38	16.0	Loss
39	38.0	Loss
40	10.0	Loss
41	37.0	Loss
42	22.0	Loss
43	34.0	Loss
44	22.0	Loss
45	19.0	Loss
46	19.0	Loss
47	12.0	Loss
48	12.0	Loss
49	18.0	Loss


In [3]:
# 匯入math函式，提供數學相關函數與常數
import math

# 定義台車行進方向，0代表往左，1代表往右
left, right = 0, 1

# 設定最大角度閾值(度數)，表示杆子相對垂直方向偏離超過多少度時，台車會往偏離的方向移動
# 例如，杆子向右傾斜超過8度，台車就往右推動；杆子向左傾斜超過8度，台車就往左推動
max_angle = 8

In [4]:
class Agent:
    # 初始化代理人
    def __init__(self):
        self.direction = left        # 當前移動方向(0:左、1:右)
        self.last_direction = right  # 上一次的移動方向，初始為右(防止一開始就重複)

    # 根據觀測值決定下一步動作
    def act(self, observation):
        # 將觀測值拆解成變數(依序為：台車位置、台車速度、桿子角度、桿子角速度)
        cart_position, cart_velocity, pole_angle, pole_velocity = observation

        '''
        行動策略(自訂邏輯)：
        1. 如果桿子角度在 ±8 度以內(偏離不大)：每次行動交替左右移動，避免持續單一方向，讓桿子盡量維持平衡。
        2. 如果桿子角度超過 8 度(偏右)：將方向設為右，推回桿子。
        3. 如果桿子角度小於 -8 度(偏左)：將方向設為左，推回桿子。
        '''

        # 如果桿子角度在-8度到+8度之間(尚可接受的偏移範圍)
        if pole_angle < math.radians(max_angle) and pole_angle > math.radians(-max_angle):
            # 在允許角度範圍內，左右交替行動(避免持續往同一方向推)
            self.direction = (self.last_direction + 1) % 2

        # 如果桿子角度超過+8度(往右傾斜太多)
        elif pole_angle >= math.radians(max_angle):
            # 將台車往右推，以幫助桿子回正
            self.direction = right

        # 剩下情況就是桿子往左傾斜太多(小於-8度)
        else:
            # 將台車往左推，以幫助桿子回正
            self.direction = left

        # 儲存這次的行動方向，供下一次判斷是否交替使用
        self.last_direction = self.direction

        # 回傳這次要執行的動作(0 = 往左，1 = 往右)，供環境執行
        return self.direction

In [5]:
# 重置環境，取得初始觀測值與環境資訊
observation, info = env.reset()

# 初始化紀錄用變數
all_rewards = []   # 每回合的總報酬紀錄清單
all_steps = []     # 每回合的總步數紀錄清單
total_rewards = 0  # 當前回合的累積報酬
total_steps = 0    # 當前回合的累積步數
no = 50            # 設定要執行的回合數為50

# 建立自訂代理人Agent物件
agent = Agent()

# 開始執行每回合的模擬
# 當回合數大於0時繼續模擬
while no > 0:
    # 根據當前觀測值，由代理人決定行動
    action = agent.act(observation)

    # 每執行一步，步數加1
    total_steps += 1

    # 執行環境一步，並取得新觀測值、報酬、終止標記、截斷標記與資訊
    observation, reward, terminated, truncated, info = env.step(action)

    # 判斷是否為回合結束(包含成功結束或失敗結束)
    done = terminated or truncated

    # 將這一步的報酬累加到當前回合的總報酬中
    total_rewards += reward

    # 如果回合結束(done為True)
    if done:
        observation, info = env.reset()    # 重置環境，準備下一回合
        all_rewards.append(total_rewards)  # 記錄該回合總報酬
        total_rewards = 0                  # 重置報酬累計
        all_steps.append(total_steps)      # 記錄該回合總步數
        total_steps = 0                    # 重置步數累計
        no -= 1                            # 減少剩餘回合數

# 關閉環境，釋放資源
env.close()

In [6]:
# 顯示每回合的執行結果，包括回合數、累積報酬和結果判定
print('回合\t報酬\t結果')

# 判斷結果：如果步數達到或超過200，視為勝利，否則為失敗
for i, (rewards, steps) in enumerate(zip(all_rewards, all_steps)):
    result = 'Win' if steps >= 200 else 'Loss'
    
    # 輸出回合編號、累積報酬和結果
    print(f'{i}\t{rewards}\t{result}')

回合	報酬	結果
0	75.0	Loss
1	89.0	Loss
2	94.0	Loss
3	97.0	Loss
4	71.0	Loss
5	102.0	Loss
6	112.0	Loss
7	97.0	Loss
8	105.0	Loss
9	89.0	Loss
10	73.0	Loss
11	48.0	Loss
12	77.0	Loss
13	75.0	Loss
14	89.0	Loss
15	82.0	Loss
16	108.0	Loss
17	51.0	Loss
18	77.0	Loss
19	82.0	Loss
20	77.0	Loss
21	58.0	Loss
22	75.0	Loss
23	94.0	Loss
24	82.0	Loss
25	58.0	Loss
26	48.0	Loss
27	93.0	Loss
28	77.0	Loss
29	79.0	Loss
30	68.0	Loss
31	154.0	Loss
32	83.0	Loss
33	53.0	Loss
34	96.0	Loss
35	97.0	Loss
36	69.0	Loss
37	121.0	Loss
38	75.0	Loss
39	93.0	Loss
40	78.0	Loss
41	97.0	Loss
42	100.0	Loss
43	110.0	Loss
44	142.0	Loss
45	74.0	Loss
46	108.0	Loss
47	48.0	Loss
48	67.0	Loss
49	120.0	Loss


In [7]:
# 匯入numpy套件，使用數學運算及陣列操作
import numpy as np

# 定義函式play，接受兩個參數：環境env和策略policy
def play(env, policy):
    # 重置環境，取得初始觀測值與資訊
    observation, info = env.reset()

    done = False       # 初始化回合狀態為未結束
    score = 0          # 紀錄總得分
    observations = []  # 儲存所有觀測值(狀態)

    # 最多執行5000步，避免無限迴圈
    for _ in range(5000):
        # 將每一步的觀測值存入observations(轉成list方便儲存)
        observations += [observation.tolist()]

        # 如果回合結束(桿子倒下或時間到)，就跳出迴圈
        if done:
            break

        # 根據策略進行動作選擇(策略是一個向量)
        outcome = np.dot(policy, observation)  # 將策略與觀測值作內積，決定行動方向
        action = 1 if outcome > 0 else 0       # 若結果>0就往右(1)，否則往左(0)

        # 執行動作，觸發下一步，並獲得新的觀測值與狀態資訊
        observation, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated  # 若已終止或截斷則表示回合結束
        score += reward                 # 將當前步驟的報酬累加到總分

    # 回傳總得分與所有觀測紀錄
    return score, observations

In [8]:
# 匯入numpy套件，使用數學運算及陣列操作
import numpy as np

# 初始化max為0，用來記錄目前最高分數、對應的觀察紀錄和策略
max = (0, [], [])

# 重複執行10次，進行10回合訓練
for _ in range(10):
    # 產生一組長度為4的隨機策略向量，元素為介於[0, 1)的浮點數
    policy = np.random.rand(1, 4)

    # 用該策略開始遊戲，取得分數和觀察紀錄
    score, observations = play(env, policy)

    # 若本回合得分高於目前最高分數，則更新最高分數與相關資料
    if score > max[0]:
        max = (score, observations, policy)

# 輸出最高得分
print('Max Score:', max[0])

Max Score: 500.0


In [9]:
# 初始化max為0，用來記錄目前最高分數、對應的觀察紀錄和策略
max = (0, [], [])

# 重複執行100次，進行100回合訓練
for _ in range(100):
    # 產生一組長度為4的隨機策略向量，元素為介於[-0.5, 0.5)的浮點數
    policy = np.random.rand(1, 4) - 0.5

    # 用該策略開始遊戲，取得分數和觀察紀錄
    score, observations = play(env, policy)

    # 若本回合得分高於目前最高分數，則更新最高分數與相關資料
    if score > max[0]:
        max = (score, observations, policy)

# 輸出最高得分
print('Max Score:', max[0])

Max Score: 500.0


## 以最大分數的policy進行實驗，驗證最佳策略是否有效

In [10]:
# 取得目前最高分數對應的最佳策略(policy)
policy = max[2]  # 從max元組中取出最佳策略，索引2為策略
policy           # 顯示最佳策略的內容

array([[ 0.1620046 , -0.16096246,  0.43499705,  0.10534346]])

## 以最佳策略取代隨機policy，進行10回合驗證    

In [11]:
# 重複執行10次，使用相同的最佳策略測試遊戲表現
for _ in range(10):
    # 用該策略開始遊戲，取得分數和觀察紀錄
    score, observations = play(env, policy)

    # 輸出每次遊戲的得分
    print('Score: ', score)

Score:  500.0
Score:  136.0
Score:  500.0
Score:  149.0
Score:  500.0
Score:  106.0
Score:  500.0
Score:  143.0
Score:  500.0
Score:  500.0
